# 解析
当我们爬取到页面的源代码后，我们不是需要页面的所有内容，我们只需要需要页面里面我们需要的数据，这里就涉及到了解析，将我们要的内容从源代码中解析出来

## XPath
学习xpath的准备：</br>
1. 安装xpath插件---在浏览器的插件中安装
2. 安装lxml库 `pip install lxml ‐i https://pypi.tuna.tsinghua.edu.cn/simple`
3. 导入lxml.etree `from lxml import etree`

xpath的使用就是一种语法，去查询我们页面中的数据，很想当时学习的CSS选择器的功能。

基本语法格式如下：
1. 路径查询</br>
    `//`：查找所有子孙节点，不考虑层级关系</br>
    `/` ：找直接子节点</br>
2. 谓词查询</br>
    `//div[@id]`</br>
    `//div[@id="maincontent"]`</br>
3. 属性查询</br>
    `//@class`</br>
4. 模糊查询</br>
    `//div[contains(@id, "he")]`</br>
    `//div[starts‐with(@id, "he")]`</br>
5. 内容查询</br>
    `//div/h1/text()`</br>
6. 逻辑运算</br>
    `//div[@id="head" and @class="s_down"]`</br>
    `//title | //price`</br>

这里的案例练习，我们先用本地的HTML文件来练习，后面再在实战中去使用

### 本地文件Xpath解析
`etree.parse()`:解析本地文件</br>

    html_tree = etree.parse('XX.html')
    html_tree.xpath(xpath路径)

In [1]:
from lxml import etree

In [9]:
# 1. xpath解析本地文件
tree = etree.parse("./data/本地文件-xpath的使用文件.html")

# 查找ul下面的li
li_list = tree.xpath("//body/ul/li")
li_list

[<Element li at 0x22de8ae77c0>,
 <Element li at 0x22de8ada440>,
 <Element li at 0x22de8ae0740>,
 <Element li at 0x22de8adb400>,
 <Element li at 0x22de8adb380>,
 <Element li at 0x22de8adb2c0>,
 <Element li at 0x22de8adb6c0>]

In [11]:
# 查找所有有id的属性的li标签的内容
li_id_list = tree.xpath('//ul/li[@id]/text()')
li_id_list

['北京', '上海', '深圳', '武汉']

In [17]:
# 找到id为li的l1标签的内容，注意在使用过程的引号问题
li_id_l1_list = tree.xpath('//ul/li[@id="l1"]/text()')  # 这里注意id = "l1" 有引号
li_id_l1_list

['北京']

In [18]:
# 查找到id为l1的li标签的class的属性值
li_id_l1_class_list = tree.xpath('//ul/li[@id="l1"]/@class')
li_id_l1_class_list

['c1']

上述的几种方式是我们最常用的，下面的跟我们学习的css中的属性选择器差不多

In [21]:
# 查询id中包含l的li标签
li_id_l_list = tree.xpath('//ul/li[contains(@id, "l")]/text()')
li_id_l_list

['北京', '上海']

In [23]:
# 查询id的值以c开头的li标签
li_id_c_list = tree.xpath('//ul/li[starts-with(@id,"c")]/text()')
li_id_c_list

['深圳', '武汉']

In [26]:
#查询id为l1和class为c1的
li_list_1 = tree.xpath('//ul/li[@id="l1" and @class="c1"]/text()')  # 这个表示li中的情况二选一

li_list_2 = tree.xpath('//ul/li[@id="l1"]/text() | //ul/li[@id="l2"]/text()')  # 这里表示选择器两种情况二选一
li_list_1, li_list_2

(['北京'], ['北京', '上海'])

### 服务器响应的数据解析
    tree = etree.HTML(content)
    result = tree.xpath()

实例：获取百度页面中的百度一下

In [27]:
import urllib.request

In [28]:
url = "https://www.baidu.com"
headers = {
     'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
}

In [36]:
# 请求对象的定制
request = urllib.request.Request(url=url, headers=headers)

# 模拟浏览器向服务器发送请求
response = urllib.request.urlopen(request)

# 获取网页源码
content = response.read().decode('utf-8')

# 解析网页源码，来获取我们想要的数据
from lxml import etree
tree = etree.HTML(content)

# 获取想要的数据
result_list = tree.xpath('//input[@id="su"]/@value')
result = result_list[0]
result

'百度一下'

实现爬取页面中的指定数据-爬取站长素材中的照片

1. 请求对象的定制
2. 获取网页的源码
3. 解析想要的数据并下载

In [6]:
import urllib.request
from lxml import etree

In [15]:
# （1）请求对象的定制
def create_request_zz(page):
    if(page == 1):
        url = 'https://sc.chinaz.com/tupian/fengjing.html'
    else:
        url = 'https://sc.chinaz.com/tupian/fengjing_' + str(page) + '.html'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    }

    request = urllib.request.Request(url = url, headers = headers)
    return request

# 模拟浏览器发送请求
def get_content_zz(request):
    response = urllib.request.urlopen(request)
    content = response.read().decode('utf-8')
    return content

# 下载数据
def down_load_zz(content):
#     下载图片
    # urllib.request.urlretrieve('图片地址','文件的名字')
    tree = etree.HTML(content)

    name_list = tree.xpath('//img[@class="lazy"]/@alt')

    # 一般设计图片的网站都会进行懒加载，所以我们不能直接获取scr的信息
    src_list = tree.xpath('//img[@class="lazy"]/@data-original')
    # 测试Xpath解析效果
    # print(name_list)
    # print(src_list)

    for i in range(len(name_list)):
        name = name_list[i]
        src = src_list[i]
        # 确保 URL 是完整的
        url = 'https:' + src
        # 测试是否获取正确的url，
        # print(url)
        urllib.request.urlretrieve(url=url,filename='./data/风景图片/' + name + '.jpg')


# 程序入口，书写逻辑
start_page = int(input('请输入起始页码'))
end_page = int(input('请输入结束页码'))

for page in range(start_page,end_page+1):
    # (1) 请求对象的定制
    request = create_request_zz(page)
    # （2）获取网页的源码
    content = get_content_zz(request)
    # （3）下载
    down_load_zz(content)

请输入起始页码 1
请输入结束页码 10


['阿尔卑斯山湖泊山水风景图片', '小清新海边沙滩海浪浪花摄影图片', '蓝色大海沙滩海浪唯美摄影图片', '绿色草地野生驴吃草图片', '唯美海浪沙滩海边摄影图片', '云雾缭绕阿尔卑斯山脉摄影图片', '唯美黄昏渔人码头摄影图片', '巍峨峡谷山脉湖泊山水风景图片', '绿色原野蓝色天空黄色热气球图片', '蓝天白云绿色农场草地风车房摄影图片', '夏日蓝色天空卷积云海边图片', '秋天枯黄草原木栅栏风景图片', '秋天氛围感公园风景摄影图片', '一望无际荒漠风景摄影图片', '河边绿色树木树叶摄影图片', '乡村龙脊梯田风景摄影图片', '挪威海峡风景摄影图片', '冬季山脉树林积雪覆盖图片', '蔚蓝色天空白色卷积云摄影图片', '唯美秋天氛围感枫叶摄影图片', '日暮黄昏大海夕阳晚霞摄影图片', '阳光草地绿色森林风景摄影图片', '蓝天白云巍峨雪山山脉图片', '日暮黄昏大海晚霞摄影图片', '秋天氛围感森林雾气电缆车风景图片', '日暮黄昏大海夕阳西下摄影图片', '冬季攀登雪山之巅摄影图片', '蓝色天空户外背包旅行登山摄影图片', '日暮黄昏大海夕阳红摄影图片', '夏日午后黄昏草甸夕阳落日余晖摄影图片', '蓝色大海浪花沙滩鸟瞰图摄影', '唯美日落黄昏夕阳余晖湖泊摄影图片', '唯美落日余晖海边风景摄影图片', '初冬山水湖泊风景摄影图片', '日暮黄昏远山山脉夕阳西下图片', '唯美山头日出山水风景摄影图片', '唯美日暮黄昏夕阳鸿鹄摄影图片', '紫色黄昏唯美湖泊风景图片', '黄昏海岸夕阳风光摄影图片', '日暮黄昏巍峨雪山山脉风光摄影图片']
https://scpic2.chinaz.net/files/default/imgs/2024-09-12/d9abcf602aa3ed05_s.jpg
https://scpic2.chinaz.net/files/default/imgs/2024-09-13/4bdfb46dada69f49_s.jpg
https://scpic2.chinaz.net/files/default/imgs/2024-09-13/fd1f734eb7e439c8_s.jpg
https://scpic2.chinaz.net/files/default/imgs/2024-09-12/173a5e4

通过上述方式，我们可以获取页面指定页数的图片信息。

注意事项：就是懒加载，我们获取图片的地址时，不一定时src，有可能是其它的，所以我们需要观察好，a标签中是否有其它属性存放的url地址。

我们这里使用`urllib.request.urlretrieve(url=url,filename=filename)`来通过指定url下载文件到指定的本地文件，非常简单，可以用于一些没有反爬虫的网站，但是无法自定义请求头，接下来我们讲述一个有反爬虫机制的网站，需要定制请求对象的

完整实现爬取页面中的指定数据----爬取菜鸟图库的照片

In [2]:
import urllib.request
from lxml import etree

In [5]:
# 1.请求对象的定制
def create_request(page):
    if(page == 1):
        url = "https://www.sucai999.com/searchlist/25105.html"
        headers = {
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
        'Cookie':'f=; v=; sid=; Hm_lvt_e57c2e187cc1668bba7f86d1784c0298=1726501585; HMACCOUNT=B6B9B0E2B71048A3; Hm_lpvt_e57c2e187cc1668bba7f86d1784c0298=1726502355',
        'Referer': 'https://www.sucai999.com/searchlist/25105.html'
    }
    else:
        url = "https://www.sucai999.com/searchlist/25105-" +str(page)+ ".html"
        headers = {
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
        'Cookie':'f=; v=; sid=; Hm_lvt_e57c2e187cc1668bba7f86d1784c0298=1726501585; HMACCOUNT=B6B9B0E2B71048A3; Hm_lpvt_e57c2e187cc1668bba7f86d1784c0298=1726502355',
        'Referer': 'https://www.sucai999.com/searchlist/25105-' + str(page) + '.html'
    }

    request = urllib.request.Request(url=url, headers=headers)
    return request

# 2. 获取网页中的源码
def get_content(request):
    response = urllib.request.urlopen(request)
    content = response.read().decode('utf-8')
    return content

# 3.解析想要的数据并下载
def down_load(page, content):
    tree = etree.HTML(content)
    name_list = tree.xpath('//ul[@id="flow"]//a/img/@alt')

    # 一般情况下设计图片的网站会进行懒加载，但是这里我们并没有进行懒加载，如果是懒加载，我们需要往后面滑动，看看src是否改变
    src_list = tree.xpath('//ul[@id="flow"]//a/img/@data-src')

    for i in range(len(name_list)):
        name = name_list[i]
        src = src_list[i]
        # 确保 URL 是完整的
        if not src.startswith('https:'):
            url = 'https:' + src
        else:
            url = src

        # 创建带请求头的 Request 对象
        if(page == 1):
            headers = {
            'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
            'Cookie':'f=; v=; sid=; Hm_lvt_e57c2e187cc1668bba7f86d1784c0298=1726501585; HMACCOUNT=B6B9B0E2B71048A3; Hm_lpvt_e57c2e187cc1668bba7f86d1784c0298=1726502355',
            'Referer': 'https://www.sucai999.com/searchlist/25105.html'
            }
        else:
            headers = {
            'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
            'Cookie':'f=; v=; sid=; Hm_lvt_e57c2e187cc1668bba7f86d1784c0298=1726501585; HMACCOUNT=B6B9B0E2B71048A3; Hm_lpvt_e57c2e187cc1668bba7f86d1784c0298=1726502355',
            'Referer': 'https://www.sucai999.com/searchlist/25105-' + str(page) + '.html'
        }
        req = urllib.request.Request(url=url, headers=headers)
        
        # 下载图片
        try:
            with urllib.request.urlopen(req) as response:
                # 保存文件
                with open(f'./data/炫酷背景/{page}_{name}.jpg', 'wb') as file:
                    file.write(response.read())
            print(f'下载成功：{name}.jpg')
        except Exception as e:
            print(f'下载失败：{name}.jpg，错误信息：{e}')

# 程序入口，书写逻辑
start_page = int(input('请输入起始页码'))
end_page = int(input('请输入结束页码'))

for page in range(start_page,end_page+1):
    # (1) 请求对象的定制
    request = create_request(page)
    # （2）获取网页的源码
    content = get_content(request)
    # （3）下载
    down_load(page, content)

请输入起始页码 1
请输入结束页码 10


下载成功：炫酷背景图片.jpg
下载成功：儿童橙色蘑菇炫酷背景.jpg
下载成功：儿童橙色蘑菇炫酷背景.jpg
下载成功：儿童橙色蘑菇炫酷背景.jpg
下载成功：线光酷炫墨黑色背景实用广告炫酷背景.jpg
下载成功：2016炫酷字体炫酷背景.jpg
下载成功：天猫汽车节炫酷几何立体块炫酷背景banner.jpg
下载成功：科技背景  炫酷背景.jpg
下载成功：浪漫背景炫酷背景.jpg
下载成功：炫酷背景.jpg
下载成功：炫酷背景.jpg
下载成功：炫酷背景.jpg
下载成功：炫酷背景.jpg
下载成功：炫酷背景.jpg
下载成功：炫酷背景.jpg
下载成功：炫酷背景.jpg
下载成功：炫酷背景.jpg
下载成功：科技背景 炫酷背景图片.jpg
下载成功：科技背景 炫酷背景图片.jpg
下载成功：炫酷背景banner.jpg
下载成功：炫酷背景 光斑背景.jpg
下载成功：炫酷背景banner.jpg
下载成功：炫酷背景.jpg
下载成功：炫酷背景.jpg
下载成功：炫酷背景.jpg
下载成功：时尚炫酷背景.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：流线炫酷背景.jpg
下载成功：彩色炫酷背景.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景高清.jpg
下载成功：炫酷背景高清.jpg
下载成功：黑色banner背景 炫酷背景图片.jpg
下载成功：几何炫酷背景.jpg
下载成功：几何炫酷背景.jpg
下载成功：夏日炫酷背景.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：红色炫酷背景.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景高清.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景素材.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景高清.jpg
下载成功：炫酷背景图片.jpg
下载成功：炫酷背景高清.jpg
下载成功：卡通炫酷背景蓝色背景.jpg
下载成功：炫酷

这里我们就不能使用`urllib.request.urlretrieve(url=url,filename=filename)`了，因为这个去访问图片的url需要请求头的信息，我们要定制请求头信息，然后通过我们手动下载，图片也是二进制的数据，我们使用`reponse.read()`读写进入文件即可

## jsonpath
jsonpath主要是xpath在json的应用
学习jsonpath的准备：</br>

1. 安装jsonpath库 `pip install jsonpath ‐i https://pypi.tuna.tsinghua.edu.cn/simple`
2. 导入jsonpath库 `import jsonpath`

jsonpath的一些基本语法，与xpath的对比
| **XPath** | **JSONPath**     | **Description**                                              |
| --------- | ---------------- | ------------------------------------------------------------ |
| /         | $                | 表示根元素                                                   |
| .         | @                | 当前元素                                                     |
| /         | . or []          | 子元素                                                       |
| ..        | -                | 父元素                                                       |
| //        | ..               | 递归下降，JSONPath是从E4X借鉴的。                            |
| *         | *                | 通配符，表示所有的元素                                       |
| @         | -                | 属性访问字符                                                 |
| []        | []               | 子元素操作符                                                 |
| \|        | [,]              | 连接操作符在XPath 结果合并其它结点集合。JSONP允许name或者数组索引。 |
| -         | [start:end:step] | 数组分割操作从ES4借鉴。                                      |
| []        | ?()              | 应用过滤表示式                                               |
| -         | ()               | 脚本表达式，使用在脚本引擎下面。                             |
| ()        | -                | Xpath分组                                                    |


In [4]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple jsonpath

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for jsonpath: filename=jsonpath-0.82.2-py3-none-any.whl size=5628 sha256=6c84c826f6a08bce7c8b638043f271b7ed827818f410df73c02547ab2c19b6c8
  Stored in directory: c:\users\ycz\appdata\local\pip\cache\wheels\d3\05\3b\b1cb0348ee9bdf569a25c1888e7aa82d333cf1da4133aeeab7
Successfully built jsonpath


### 本地文件使用jsonpath解析

In [5]:
import json
import jsonpath

In [6]:
obj = json.load(open('./data/本地文件-jsonpath的使用文件.json', 'r', encoding='utf-8'))

In [7]:
obj

{'store': {'book': [{'category': '修真',
    'author': '六道',
    'title': '坏蛋是怎样练成的',
    'price': 8.95},
   {'category': '修真', 'author': '天蚕土豆', 'title': '斗破苍穹', 'price': 12.99},
   {'category': '修真',
    'author': '唐家三少',
    'title': '斗罗大陆',
    'isbn': '0-553-21311-3',
    'price': 8.99},
   {'category': '修真',
    'author': '南派三叔',
    'title': '星辰变',
    'isbn': '0-395-19395-8',
    'price': 22.99}],
  'bicycle': {'author': '老马', 'color': '黑色', 'price': 19.95}}}

In [8]:
# 接下来我们通过一些实例来解析json中的数据
# 获取书店所有书的作者
author_list = jsonpath.jsonpath(obj, '$.store.book[*].author')
author_list

['六道', '天蚕土豆', '唐家三少', '南派三叔']

In [10]:
# 获取所有作者
author_list_1 = jsonpath.jsonpath(obj, '$..author')
author_list_1

['六道', '天蚕土豆', '唐家三少', '南派三叔', '老马']

In [14]:
# store下面的所有的元素
tag_list = jsonpath.jsonpath(obj, '$.store.*')
tag_list

[[{'category': '修真', 'author': '六道', 'title': '坏蛋是怎样练成的', 'price': 8.95},
  {'category': '修真', 'author': '天蚕土豆', 'title': '斗破苍穹', 'price': 12.99},
  {'category': '修真',
   'author': '唐家三少',
   'title': '斗罗大陆',
   'isbn': '0-553-21311-3',
   'price': 8.99},
  {'category': '修真',
   'author': '南派三叔',
   'title': '星辰变',
   'isbn': '0-395-19395-8',
   'price': 22.99}],
 {'author': '老马', 'color': '黑色', 'price': 19.95}]

In [16]:
# store里面所有的price
price_list = jsonpath.jsonpath(obj, '$.store..price')
price_list

[8.95, 12.99, 8.99, 22.99, 19.95]

In [17]:
# 第三本书
book = jsonpath.jsonpath(obj, '$..book[2]')
book

[{'category': '修真',
  'author': '唐家三少',
  'title': '斗罗大陆',
  'isbn': '0-553-21311-3',
  'price': 8.99}]

In [20]:
# 最后一本书
book = jsonpath.jsonpath(obj, '$..book[(@.length- 1)]')
book

[{'category': '修真',
  'author': '南派三叔',
  'title': '星辰变',
  'isbn': '0-395-19395-8',
  'price': 22.99}]

In [24]:
# 前面的两本书
book_list = jsonpath.jsonpath(obj, '$..book[0:2]')
book_list_2 = jsonpath.jsonpath(obj, '$..book[:2]')
book_list, book_list_2

([{'category': '修真', 'author': '六道', 'title': '坏蛋是怎样练成的', 'price': 8.95},
  {'category': '修真', 'author': '天蚕土豆', 'title': '斗破苍穹', 'price': 12.99}],
 [{'category': '修真', 'author': '六道', 'title': '坏蛋是怎样练成的', 'price': 8.95},
  {'category': '修真', 'author': '天蚕土豆', 'title': '斗破苍穹', 'price': 12.99}])

In [25]:
# 条件过滤
# 条件过滤需要在（）的前面添加一个？
# 过滤出所有的包含isbn的书
book_list = jsonpath.jsonpath(obj, '$..book[?(@.isbn)]')
book_list

[{'category': '修真',
  'author': '唐家三少',
  'title': '斗罗大陆',
  'isbn': '0-553-21311-3',
  'price': 8.99},
 {'category': '修真',
  'author': '南派三叔',
  'title': '星辰变',
  'isbn': '0-395-19395-8',
  'price': 22.99}]

In [26]:
# 哪本书超过了10块钱
book_list = jsonpath.jsonpath(obj, '$..book[?(@.price > 10)]')
book_list

[{'category': '修真', 'author': '天蚕土豆', 'title': '斗破苍穹', 'price': 12.99},
 {'category': '修真',
  'author': '南派三叔',
  'title': '星辰变',
  'isbn': '0-395-19395-8',
  'price': 22.99}]

### 服务器响应的数据解析

使用jsonpath解析淘票票里的地址信息

In [27]:
import urllib.request
import jsonpath

In [28]:
url = "https://dianying.taobao.com/cityAction.json?activityId&_ksTS=1726583062377_108&jsoncallback=jsonp109&action="\
        "cityAction&n_s=new&event_submit_doGetAllRegion=true"
url

'https://dianying.taobao.com/cityAction.json?activityId&_ksTS=1726583062377_108&jsoncallback=jsonp109&action=cityAction&n_s=new&event_submit_doGetAllRegion=true'

In [32]:
headers = {
    # ":authority": "dianying.taobao.com",
    # ":method": "GET",
    # ":path": "/cityAction.json?activityId&_ksTS=1726583062377_108&jsoncallback=jsonp109&action=cityAction&n_s=new&event_submit_doGetAllRegion=true",
    # ":scheme": "https",
    "Accept": "text/javascript, application/javascript, application/ecmascript, application/x-ecmascript, */*; q=0.01",
    # "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en,zh-CN;q=0.9,zh;q=0.8",
    "Bx-V": "2.5.20",
    "Cookie": "cna=wqLcHGPB/UACAd9jwBi2Qxpc; miid=1382043092868146837; t=96c72763de584d2263e721e301829105; tfstk=fbQnaNX-6M-Q_Mh8KNYQVJ5eC7qOOvTWdT3JeUpzbdJ6ypQdvgjGGTvLNkTdjLXNIp-LO6Kor_v6ypQdvgjGK9uJvTQlf4ffNwFByWTCOUTzH-Uvr61BPZA8Mx2O_7R5VDvunrCCOflnUSIak0YRadVHUT-ebFRXQDuPU3zasQR-L0uFU1PMNdRrzHoU7fRpTLlE9HoeHaSV7JVkSSTC2t7MthrATdoHHNAHzK5F-4ur0BxH36JgHd4bkh5WqaGSrHCFcTOVLvyHz1AcnIX32b9hbsQd464g3eQCTi-Pu8iPneXk0w-ia4fBrp8M_ZP-cdQMpT7HbS3D2F7v0eSTfJKJ-IXPR9cozt5OMwt1r-DwFMdX7BXzuRjP1coqdpuW_7QiV0te1CvbTDAud2URihFgsmCWYCOXHaFif7xe1BaUs5mASHR6gc5..; cookie2=1e644f1259d35b7bede826a0bcfda17f; v=0; _tb_token_=ee8d38e95eee3; xlly_s=1; tb_city=110100; tb_cityName=\"sbG+qQ==\"; isg=BLCw7Vzpolr323xH22UHt0bogX4C-ZRDs7w2D6oB8IveZVAPUgh101ZbvWUFcEwb",
    "Priority": "u=1, i",
    "Referer": "https://dianying.taobao.com/?spm=a1z21.3046609.city.1.2b2c112aqecTlo&city=110100",
    "Sec-Ch-Ua": "\"Chromium\";v=\"124\", \"Google Chrome\";v=\"124\", \"Not-A.Brand\";v=\"99\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

In [34]:

request = urllib.request.Request(url=url, headers=headers)

response = urllib.request.urlopen(request)
content = response.read().decode('utf-8')
content

'\r\n\r\njsonp109({"returnCode":"0","returnValue":{"A":[{"id":3643,"parentId":0,"regionName":"阿坝","cityCode":513200,"pinYin":"ABA"},{"id":3090,"parentId":0,"regionName":"阿克苏","cityCode":652900,"pinYin":"AKESU"},{"id":3632,"parentId":0,"regionName":"阿拉善","cityCode":152900,"pinYin":"ALASHAN"},{"id":3675,"parentId":0,"regionName":"阿勒泰","cityCode":654300,"pinYin":"ALETAI"},{"id":899,"parentId":0,"regionName":"安康","cityCode":610900,"pinYin":"ANKANG"},{"id":196,"parentId":0,"regionName":"安庆","cityCode":340800,"pinYin":"ANQING"},{"id":758,"parentId":0,"regionName":"鞍山","cityCode":210300,"pinYin":"ANSHAN"},{"id":388,"parentId":0,"regionName":"安顺","cityCode":520400,"pinYin":"ANSHUN"},{"id":454,"parentId":0,"regionName":"安阳","cityCode":410500,"pinYin":"ANYANG"}],"B":[{"id":3633,"parentId":0,"regionName":"白城","cityCode":220800,"pinYin":"BAICHENG"},{"id":356,"parentId":0,"regionName":"百色","cityCode":451000,"pinYin":"BAISE"},{"id":5891,"parentId":0,"regionName":"白沙","cityCode":469025,"pinYin":"BAIS

我得到了这一堆json格式的数据,但是发现它不止json格式的数据,还有一些其他的,我们需要删除

In [35]:
content_json = content.split('(')[1].split(')')[0]
content_json

'{"returnCode":"0","returnValue":{"A":[{"id":3643,"parentId":0,"regionName":"阿坝","cityCode":513200,"pinYin":"ABA"},{"id":3090,"parentId":0,"regionName":"阿克苏","cityCode":652900,"pinYin":"AKESU"},{"id":3632,"parentId":0,"regionName":"阿拉善","cityCode":152900,"pinYin":"ALASHAN"},{"id":3675,"parentId":0,"regionName":"阿勒泰","cityCode":654300,"pinYin":"ALETAI"},{"id":899,"parentId":0,"regionName":"安康","cityCode":610900,"pinYin":"ANKANG"},{"id":196,"parentId":0,"regionName":"安庆","cityCode":340800,"pinYin":"ANQING"},{"id":758,"parentId":0,"regionName":"鞍山","cityCode":210300,"pinYin":"ANSHAN"},{"id":388,"parentId":0,"regionName":"安顺","cityCode":520400,"pinYin":"ANSHUN"},{"id":454,"parentId":0,"regionName":"安阳","cityCode":410500,"pinYin":"ANYANG"}],"B":[{"id":3633,"parentId":0,"regionName":"白城","cityCode":220800,"pinYin":"BAICHENG"},{"id":356,"parentId":0,"regionName":"百色","cityCode":451000,"pinYin":"BAISE"},{"id":5891,"parentId":0,"regionName":"白沙","cityCode":469025,"pinYin":"BAISHA"},{"id":634,"p

In [36]:
with open('./data/淘票票地址信息.json', 'w', encoding='utf-8') as fp:
    fp.write(content_json)

In [39]:
# 写入之后,我们就可以使用jsonpath进行解析
import json
import jsonpath
obj = json.load(open('./data/淘票票地址信息.json', 'r', encoding='utf-8'))
obj

{'returnCode': '0',
 'returnValue': {'A': [{'id': 3643,
    'parentId': 0,
    'regionName': '阿坝',
    'cityCode': 513200,
    'pinYin': 'ABA'},
   {'id': 3090,
    'parentId': 0,
    'regionName': '阿克苏',
    'cityCode': 652900,
    'pinYin': 'AKESU'},
   {'id': 3632,
    'parentId': 0,
    'regionName': '阿拉善',
    'cityCode': 152900,
    'pinYin': 'ALASHAN'},
   {'id': 3675,
    'parentId': 0,
    'regionName': '阿勒泰',
    'cityCode': 654300,
    'pinYin': 'ALETAI'},
   {'id': 899,
    'parentId': 0,
    'regionName': '安康',
    'cityCode': 610900,
    'pinYin': 'ANKANG'},
   {'id': 196,
    'parentId': 0,
    'regionName': '安庆',
    'cityCode': 340800,
    'pinYin': 'ANQING'},
   {'id': 758,
    'parentId': 0,
    'regionName': '鞍山',
    'cityCode': 210300,
    'pinYin': 'ANSHAN'},
   {'id': 388,
    'parentId': 0,
    'regionName': '安顺',
    'cityCode': 520400,
    'pinYin': 'ANSHUN'},
   {'id': 454,
    'parentId': 0,
    'regionName': '安阳',
    'cityCode': 410500,
    'pinYin': 'ANY

In [41]:
# 现在我们只想获取地址regionName这一个值
city_list = jsonpath.jsonpath(obj, '$..regionName')
city_list

['阿坝',
 '阿克苏',
 '阿拉善',
 '阿勒泰',
 '安康',
 '安庆',
 '鞍山',
 '安顺',
 '安阳',
 '白城',
 '百色',
 '白沙',
 '白山',
 '白银',
 '保定',
 '宝鸡',
 '保山',
 '包头',
 '巴彦淖尔',
 '巴中',
 '北海',
 '北京',
 '蚌埠',
 '本溪',
 '毕节',
 '滨州',
 '亳州',
 '巴音郭楞',
 '沧州',
 '长春',
 '常德',
 '昌吉',
 '昌江',
 '长沙',
 '长治',
 '常州',
 '巢湖市',
 '朝阳',
 '潮州',
 '承德',
 '成都',
 '澄迈县',
 '郴州',
 '赤峰',
 '池州',
 '重庆',
 '崇左',
 '楚雄',
 '滁州',
 '大理',
 '大连',
 '儋州',
 '丹东',
 '大庆',
 '大同',
 '大兴安岭',
 '达州',
 '屯昌',
 '德宏',
 '德阳',
 '德州',
 '定安',
 '定西',
 '迪庆',
 '东方',
 '东莞',
 '东营',
 '鄂尔多斯',
 '恩施',
 '鄂州',
 '防城港',
 '佛山',
 '抚顺',
 '阜新',
 '阜阳',
 '抚州',
 '福州',
 '甘南',
 '赣州',
 '甘孜',
 '巩义市',
 '广安',
 '广元',
 '广州',
 '贵港',
 '桂林',
 '贵阳',
 '固原',
 '哈尔滨',
 '海北',
 '海东',
 '海口',
 '海南州',
 '海西',
 '哈密',
 '韩城市',
 '邯郸',
 '杭州',
 '汉中',
 '鹤壁',
 '河池',
 '合肥',
 '鹤岗',
 '黑河',
 '衡水',
 '衡阳',
 '和田',
 '河源',
 '菏泽',
 '贺州',
 '红河',
 '淮安',
 '淮北',
 '怀化',
 '淮南',
 '黄冈',
 '黄南',
 '黄山',
 '黄石',
 '呼和浩特',
 '惠州',
 '葫芦岛',
 '呼伦贝尔',
 '湖州',
 '佳木斯',
 '吉安',
 '江门',
 '焦作',
 '嘉兴',
 '嘉峪关',
 '揭阳',
 '吉林',
 '济南',
 '金昌',
 '晋城',
 '景德镇',
 '荆门',
 '荆州',
 '金华',
 

获取boss直聘上的工作类别

In [44]:
url = "https://www.zhipin.com/wapi/zpCommon/data/getCityShowPosition?cityCode=101270700"

headers = {
    # ":authority": "www.zhipin.com",
    # ":method": "GET",
    # ":path": "/wapi/zpCommon/data/getCityShowPosition?cityCode=101270700",
    # ":scheme": "https",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    # "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en,zh-CN;q=0.9,zh;q=0.8",
    "Cache-Control": "no-cache",
    "Cookie": "__l=r=https%3A%2F%2Fcn.bing.com%2F&l=%2F&s=1; __g=-; Hm_lvt_194df3105ad7148dcf2b98a91b5e727a=1726585220; HMACCOUNT=B6B9B0E2B71048A3; __zp_seo_uuid__=f3d662fa-4a29-438d-99e3-3286c24c3cdf; __c=1726585220; __a=80516006.1687618435.1717410977.1726585220.73.6.22.73; __zp_stoken__=498bfw4JuO1k8Exd%2BZ1J1wrtCw4FWXhPCvE5JXWXDhHNhTsK7WMKGwplIwplWwr9owq5QT8OBVWXCisK%2Bw7rCrsSEWMSAwrXCjcKww7NfwqXCr8SHVcOYwqDEgMK8w53CocKVwo3DucKow4vCmcOhwojDmcKrw6zCr8OWwpfDmVDFh8K2xIHDvMS0wr%2FEr8KXxKnCmcOYwqPEscOHxKbDh8SIwrXDk8WvxJzGhMWbxYXEp8K4wqY9NQ0ICQgIFxYTFhYRFAoXFwsCDwICDQgJCAg%2BNMO5xITDjjgyQDguSlNWDkplZUllTg9fTkk4OWFgAl85MznDhsKOPTjDhMOKw4Fjwro5w4FIw4M4w4DCmDhFODnCv8KkLinCvBEMNgxGOMK9LAzCucO5DMONZsOkwpnCoMK4xL41MjLCvcS7QD0eRz08Qz04QDw9LjgTw4lmw6XCnMKgwr81LT0dRD09Qz88PT1BPUYpPUAsLj04LUERDQ8KFS9GwrxXwrjDoD09; Hm_lpvt_194df3105ad7148dcf2b98a91b5e727a=1726586281; lastCity=101270700",
    "Pragma": "no-cache",
    "Priority": "u=1, i",
    "Referer": "https://www.zhipin.com/chengshi/c101270700/?seoRefer=index",
    "Sec-Ch-Ua": "\"Chromium\";v=\"124\", \"Google Chrome\";v=\"124\", \"Not-A.Brand\";v=\"99\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "Traceid": "F-91412fVbNEWlyn8y",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}


In [45]:
request = urllib.request.Request(url=url, headers=headers)
response = urllib.request.urlopen(request)
content = response.read().decode()
content

'{"code":0,"message":"Success","zpData":{"position":[{"code":1010000,"name":"互联网/AI","tip":null,"subLevelModelList":[{"code":1000020,"name":"后端开发","tip":null,"subLevelModelList":[{"code":100101,"name":"Java","tip":null,"subLevelModelList":null,"firstChar":null,"pinyin":null,"rank":0,"mark":11,"positionType":3,"cityType":0,"capital":0,"color":null,"recruitmentType":"1,2,3","cityCode":null,"regionCode":0,"value":null},{"code":100102,"name":"C/C++","tip":null,"subLevelModelList":null,"firstChar":null,"pinyin":null,"rank":0,"mark":11,"positionType":3,"cityType":0,"capital":0,"color":null,"recruitmentType":"1,2,3","cityCode":null,"regionCode":0,"value":null},{"code":100103,"name":"PHP","tip":null,"subLevelModelList":null,"firstChar":null,"pinyin":null,"rank":0,"mark":11,"positionType":3,"cityType":0,"capital":0,"color":null,"recruitmentType":"1,2,3","cityCode":null,"regionCode":0,"value":null},{"code":100109,"name":"Python","tip":null,"subLevelModelList":null,"firstChar":null,"pinyin":null,

In [46]:
with open('./data/boss直聘职位.json', 'w', encoding='utf-8') as fp:
    fp.write(content)

In [47]:
obj = json.load(open('./data/boss直聘职位.json', 'r', encoding='utf-8'))
obj

{'code': 0,
 'message': 'Success',
 'zpData': {'position': [{'code': 1010000,
    'name': '互联网/AI',
    'tip': None,
    'subLevelModelList': [{'code': 1000020,
      'name': '后端开发',
      'tip': None,
      'subLevelModelList': [{'code': 100101,
        'name': 'Java',
        'tip': None,
        'subLevelModelList': None,
        'firstChar': None,
        'pinyin': None,
        'rank': 0,
        'mark': 11,
        'positionType': 3,
        'cityType': 0,
        'capital': 0,
        'color': None,
        'recruitmentType': '1,2,3',
        'cityCode': None,
        'regionCode': 0,
        'value': None},
       {'code': 100102,
        'name': 'C/C++',
        'tip': None,
        'subLevelModelList': None,
        'firstChar': None,
        'pinyin': None,
        'rank': 0,
        'mark': 11,
        'positionType': 3,
        'cityType': 0,
        'capital': 0,
        'color': None,
        'recruitmentType': '1,2,3',
        'cityCode': None,
        'regionCode': 0,


In [57]:
# 这里我们通过jsonpath解析互联网/AI下面的职位
# wAI_list = jsonpath.jsonpath(obj, '$.zpData.position[?(@.name=="互联网/AI")].subLevelModelList[*].name')
wAI_list = jsonpath.jsonpath(obj, '$.zpData.position[?(@.name=="互联网/AI")].subLevelModelList..name')
wAI_list

['后端开发',
 'Java',
 'C/C++',
 'PHP',
 'Python',
 'C#',
 '.NET',
 'Golang',
 'Node.js',
 'Hadoop',
 '语音/视频/图形开发',
 'GIS工程师',
 '区块链工程师',
 '全栈工程师',
 '其他后端开发',
 '前端/移动开发',
 '前端开发工程师',
 'Android',
 'iOS',
 'U3D',
 'UE4',
 'Cocos',
 '技术美术',
 'JavaScript',
 '鸿蒙开发工程师',
 '测试',
 '测试工程师',
 '软件测试',
 '自动化测试',
 '功能测试',
 '测试开发',
 '硬件测试',
 '游戏测试',
 '性能测试',
 '渗透测试',
 '测试经理',
 '运维/技术支持',
 '运维工程师',
 'IT技术支持',
 '网络工程师',
 '网络安全',
 '系统工程师',
 '运维开发工程师',
 '系统管理员',
 'DBA',
 '电脑/打印机维修',
 '系统安全',
 '技术文档工程师',
 '人工智能',
 '图像算法',
 '自然语言处理算法',
 '大模型算法',
 '数据挖掘',
 '规控算法',
 'SLAM算法',
 '推荐算法',
 '搜索算法',
 '语音算法',
 '风控算法',
 '算法研究员',
 '算法工程师',
 '机器学习',
 '深度学习',
 '自动驾驶系统工程师',
 '数据标注/AI训练师',
 '销售技术支持',
 '售前技术支持',
 '售后技术支持',
 '销售技术支持',
 '客户成功',
 '数据',
 '数据分析师',
 '数据开发',
 '数据仓库',
 'ETL工程师',
 '数据挖掘',
 '数据架构师',
 '爬虫工程师',
 '数据采集',
 '数据治理',
 '技术项目管理',
 '项目经理/主管',
 '项目助理',
 '项目专员',
 '实施工程师',
 '实施顾问',
 '需求分析工程师',
 '硬件项目经理',
 '高端技术职位',
 '技术经理',
 '架构师',
 '技术总监',
 'CTO/CIO',
 '技术合伙人',
 '运维总监',
 '其他技术职位',
 '其他技术职位']

第一个实战例子相对简单,使用jsonpath的基本使用,第二个实战例子,就需要考虑获取子类中的所有内容,怎么考虑才可以获取所有内容

jsonpath是非常常用的,就目前来说,很多网站还是用ajax请求与服务器进行交互,这些请求的响应通常是JSON格式的数据.所以我们使用jsonpath解析API返回的数据,是很方便的

## BS4

BS4的使用流程
1. 安装：`pip install bs4`
2. 导入：`from bs4 import BeautifulSoup`
3. 创建对象</br>
    服务器响应的文件生成对象：`soup = BeautifulSoup(response.read().decode(), 'lxml')`</br>
    本地文件生成对象：`soup = BeautifulSoup(open('1.html'), 'lxml')`

注意：默认打开文件的编码格式gbk所以需要指定打开编码格式

In [58]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple bs4

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


BS4的使用更像我们编程的语法，使用某一个子类下面的某个属性，很像我们学习的CSS选择器和JS中的DOM

节点定位：</br>
1. 根据标签名查找节点</br>

    soup.a 【注】只能找到第一个a</br>
    soup.a.name</br>
    soup.a.attrs</br>
    
2. 函数</br>

    (1).find(返回一个对象)
    
        find('a')：只找到第一个a标签
        find('a', title='名字')
        find('a', class_='名字')

    (2).find_all(返回一个列表)
    
        find_all('a') 查找到所有的a
        find_all(['a', 'span']) 返回所有的a和span
        find_all('a', limit=2) 只找前两个a

    (3).select(根据选择器得到节点对象)【推荐】
    
        1.element
            eg:p
        2..class
            eg:.firstname
        3.#id
            eg:#firstname
        4.属性选择器
            [attribute]
                eg:li = soup.select('li[class]')
            [attribute=value]
                eg:li = soup.select('li[class="hengheng1"]')
        5.层级选择器
            element element
                div p
            element>element
                div>p
            element,element
                div,p
            eg:soup = soup.select('a,span')

节点信息：</br>
(1).获取节点内容：适用于标签中嵌套标签的结构

    obj.string
    obj.get_text()【推荐】
(2).节点的属性

    tag.name 获取标签名
        eg:tag = find('li)
            print(tag.name)
    tag.attrs将属性值作为一个字典返回
(3).获取节点属性

    obj.attrs.get('title')【常用】
    obj.get('title')
    obj['title']

### 本地文件使用bs4解析

首先我们还是去解析本地的文件，熟悉bs4的使用，后面再通过实战进行练习回顾

In [1]:
from bs4 import BeautifulSoup

In [41]:
# 默认打开的文件的编码格式是gbk 所以在打开文件的时候需要指定编码
soup = BeautifulSoup(open('./data/本地文件-bs4的使用文件.html', 'r', encoding='utf-8'), 'lxml')
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Title</title>
</head>
<body>
<div>
<ul>
<li id="l1">张三</li>
<li id="l2">李四</li>
<li>王五</li>
<a class="a1" href="" id="">遂宁职业学院</a>
<span>Hello</span>
</ul>
</div>
<a href="" title="a2">百度</a>
<div id="d1">
      div div
      <span> bs4 </span>
</div>
<p class="p1" id="p1">urllib</p>
</body>
</html>

In [4]:
# 下面我们通过一个个实例来讲述bs4的基本语法
# 1.找到第一个a标签
soup.a

<a class="a1" href="" id="">遂宁职业学院</a>

In [5]:
# 2.获取第一个a标签的属性和属性值
soup.a.attrs  # 返回的结果是一个字典类型，属性:属性值

{'href': '', 'id': '', 'class': ['a1']}

In [12]:
# bs4中一些函数的使用
# (1)find()
# 返回第一个a标签的信息
soup.find('a')

<a class="a1" href="" id="">遂宁职业学院</a>

In [14]:
# 根据属性（title）的值(a2)找到对应的a标签
soup.find('a', title='a2')  # 我们找到的都是从上至下的第一个符合条件的元素节点

<a class="a1" href="" id="">遂宁职业学院</a>

In [15]:
# 这里我们同样可以使用其它属性 比如 href class id这些属性，但是使用class时要注意参数为class_,因为class跟python的关键字撞了
soup.find('a', href='')

<a class="a1" href="" id="">遂宁职业学院</a>

In [16]:
soup.find('a', class_='a1')

<a class="a1" href="" id="">遂宁职业学院</a>

In [21]:
soup.find('a', id_='')  # 我们使用id的时候，是可以直接使用id的，但是建议使用id_

<a class="a1" href="" id="">遂宁职业学院</a>

In [22]:
# 2.find_all() 返回的结果是一个列表，返回所有符合条件的
# 查询所有a标签
soup.find_all('a')

[<a class="a1" href="" id="">遂宁职业学院</a>, <a href="" title="a2">百度</a>]

In [23]:
# 查询所有a标签和span标签
soup.find_all(['a', 'span'])  # 注意要查询多个标签的数据，我们给find_all的是一个列表

[<a class="a1" href="" id="">遂宁职业学院</a>,
 <span>Hello</span>,
 <a href="" title="a2">百度</a>,
 <span> bs4 </span>]

In [24]:
# 查询前2个span标签
soup.find_all('span', limit=2)  # limit表示查询的个数

[<span>Hello</span>, <span> bs4 </span>]

In [25]:
# 3.select方法 这个方法是最全面的，也是最推荐使用的,使用起来很像我们css选择器的使用，它返回的结果是一个列表
soup.select('a')  # 表示查询所有a标签

[<a class="a1" href="" id="">遂宁职业学院</a>, <a href="" title="a2">百度</a>]

In [26]:
# 通过class查询
soup.select('.a1')  # 表示查询所有class = a1的标签

[<a class="a1" href="" id="">遂宁职业学院</a>]

In [27]:
# 通过id查询
soup.select('#l1')  # 表示查询id为l1的标签

[<li id="l1">张三</li>]

注意：在我们写前端代码的时候，我们的id肯定是唯一的，所以只有两种结果，获取为null或获取为一个标签，但是这个标签仍然放在一个列表中

In [30]:
# 通过属性选择器查询(跟CSS完全一致)
soup.select('li[id]')  # 查询li标签中有id属性的标签

[<li id="l1">张三</li>, <li id="l2">李四</li>]

In [31]:
soup.select('li[id = "l2"]')  # 查询li标签中id = l2的标签

[<li id="l2">李四</li>]

In [32]:
# 层次选择器
soup.select('div li')  # 找到div下面的li

[<li id="l1">张三</li>, <li id="l2">李四</li>, <li>王五</li>]

In [34]:
# 子代选择器
soup.select('div > ul > li') # div 下面的ul下面的li  一层层的关系

[<li id="l1">张三</li>, <li id="l2">李四</li>, <li>王五</li>]

以上是我们对元素节点的定位，对元素节点定位完后，我们需要获取元素节点的一些信息，接下来我们讲述节点的信息

In [42]:
# 1. 获取节点内容
obj = soup.select('#d1')[0]
obj

<div id="d1">
      div div
      <span> bs4 </span>
</div>

如果元素节点中，只有属性节点和文本节点时，我们可以通过`string`和`get_text()`都可以使用</br>
如果元素节点中有其它元素节点，string就获取不到数据，而get_text()可以获取数据</br>
所以一般情况下我们使用get_text()</br>

In [43]:
# 2. 获取文本节点
obj.string  # null

In [44]:
obj.get_text()  # 这里是获取该元素节点下面的所有文本节点的内容

'\n      div div\n       bs4 \n'

In [51]:
# 3. 获取元素节点的属性节点
obj = soup.select('#p1')[0]
obj

<p class="p1" id="p1">urllib</p>

In [52]:
obj.name  # 获取该节点的标签名

'p'

In [59]:
obj.attrs  # 属性：属性值

{'id': 'p1', 'class': ['p1']}

In [61]:
# 获取属性节点的内容
obj = soup.select('#p1')[0]
obj

<p class="p1" id="p1">urllib</p>

In [62]:
obj.attrs.get('class')

['p1']

In [63]:
obj.get('class')

['p1']

In [64]:
obj['class']

['p1']

In [65]:
obj['id']

'p1'

上述是我们对于bs4的基本语法的练习，通过简单的本地文件，直观的看到bs4的语法，接下来我们将通过一个例子进行实战

### 服务器响应的数据解析

这里跟之前的模拟是一样的，我们通过获取页面的源码，来解析页面得到我们想要的

该实战我们来获取麦当劳的汉堡数据

In [101]:
import urllib.request
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [68]:
url = 'https://www.mcdonalds.com.cn/product/mcdonalds/hamburgers'

headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    # "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en,zh-CN;q=0.9,zh;q=0.8",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Cookie": "ARRAffinity=3686f4543397f687579628090250a9a8709791c6b79845b95560ab324fb0d7f7; ARRAffinitySameSite=3686f4543397f687579628090250a9a8709791c6b79845b95560ab324fb0d7f7; sajssdk_2015_cross_new_user=1; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%22192015861444fc-00784405e362fe-26001d51-1821369-192015861451462%22%2C%22%24device_id%22%3A%22192015861444fc-00784405e362fe-26001d51-1821369-192015861451462%22%2C%22props%22%3A%7B%22%24latest_referrer%22%3A%22https%3A%2F%2Fcn.bing.com%2F%22%2C%22%24latest_referrer_host%22%3A%22cn.bing.com%22%2C%22%24latest_traffic_source_type%22%3A%22%E8%87%AA%E7%84%B6%E6%90%9C%E7%B4%A2%E6%B5%81%E9%87%8F%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC%22%7D%7D; _gid=GA1.3.1833017192.1726599422; _ga_ZNNHN4PDZF=GS1.3.1726599422.1.1.1726599448.0.0.0; _ga_H0FLFLQXE1=GS1.1.1726599422.1.1.1726599493.59.0.0; _ga=GA1.3.686748742.1726599422; _gat_UA-49420844-1=1",
    "Host": "www.mcdonalds.com.cn",
    "Pragma": "no-cache",
    "Referer": "https://www.mcdonalds.com.cn/product/mcdonalds/Snacks",
    "Sec-Ch-Ua": "\"Chromium\";v=\"124\", \"Google Chrome\";v=\"124\", \"Not-A.Brand\";v=\"99\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}

In [72]:
request = urllib.request.Request(url=url, headers=headers)
response = urllib.request.urlopen(request)

content = response.read().decode('utf-8')
content

'<!DOCTYPE html>\r\n<!--[if IE 8]>\r\n<html lang="zh-CN" class="ie8"> <![endif]-->\r\n<!--[if !IE]><!-->\r\n<html lang="zh-CN">\r\n<!--<![endif]-->\r\n<head>\r\n  <meta charset="UTF-8">\r\n  <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">\r\n  <meta name="format-detection" content="telephone=no">\r\n  <meta name="x5-fullscreen" content="true">\r\n  <meta name="full-screen" content="yes">\r\n  <title>汉堡 | 麦当劳官网</title>\n<meta name=\'keywords\'\n      content="麦当劳, McDonald’s, 麦乐送, 麦咖啡, 得来速, 汉堡, McCafé, 甜品站, 优惠券, 麦当劳优惠券, 麦当劳叔叔, mcd, 麦当劳中国, 麦当当, 麦记, m记, 玩具, McDonald\'s, 金拱门, 星厨, 巨无霸, 未来2.0">\n<meta name=\'description\'\n      content="麦当劳于1990年进入中国市场，始终坚持以高品质的食品、亲切友善的服务、清洁舒适的用餐环境和物超所值为每一位顾客提供最佳的用餐体验。公司还相继引入了甜品站（1994年）、24小时营业餐厅（2005年）、得来速汽车餐厅（2006年）、麦乐送24小时送餐服务（2008年）及McCafé连锁咖啡品牌（2009年）。">\n<meta name="twitter:card" content="summary_large_image">\n<meta name="twitter:title"\n      content="汉堡 | 麦当劳官网">\n<meta name="twitter:image" content="https://offi

In [93]:
soup = BeautifulSoup(content, 'lxml')
soup_1 =soup.select('.row')[1]
name_list = soup_1.select('span[class="name"]')
src_list = soup_1.select('img')

In [104]:
for i in range(len(name_list)):
    name = name_list[i].get_text()
    src = src_list[i].get("src")
    # print(name,src)  测试
    
    # 对 URL 进行编码
    url = quote(src, safe=':/')
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    }
    request = urllib.request.Request(url=url, headers=headers)
    try:
        with urllib.request.urlopen(request) as response:
            # 保存文件
            with open(f'./data/麦当劳汉堡图片/{name}.jpg', 'wb') as file:
                file.write(response.read())
        print(f'下载成功：{name}.jpg')
    except Exception as e:
        print(f'下载失败：{name}.jpg，错误信息：{e}')
        

下载成功：巨无霸.jpg
下载成功：汉堡包.jpg
下载成功：麦辣鸡腿汉堡.jpg
下载成功：原味板烧鸡腿堡.jpg
下载成功：麦香鸡.jpg
下载成功：麦香鱼.jpg
下载成功：吉士汉堡包.jpg
下载成功：双层吉士汉堡.jpg
下载成功：不素之霸双层牛堡.jpg
下载成功：双层深海鳕鱼堡.jpg
下载成功：安格斯MAX厚牛培根堡.jpg
下载成功：安格斯MAX厚牛芝士堡.jpg
下载成功：双层安格斯MAX厚牛培根堡.jpg
下载成功：双层安格斯MAX厚牛芝士堡.jpg
下载成功：培根蔬萃双层牛堡.jpg


接下来我们可以尝试爬取其它的网站，然后解析

## 总结
一般情况下我们都会使用xpath，获取我们想要的数据，然后进行数据分析，但是有时候我们获取json数据对json数据进行解析就需要使用到jsonpath，而bs4基本上跟xpath类似，所以推荐使用xpath，bs4只是跟接近我们之前学过的语法，上手起来很快，但是还是建议使用xpath。

XPath

优点

1. **强大而灵活**：
   - `XPath` 是一种强大的语言，可以使用路径表达式、谓语、逻辑运算、条件语句等复杂的查询来定位 XML/HTML 文档中的元素。它可以轻松地处理复杂的选择条件和多层嵌套结构。
2. **支持多种选择方式**：
   - `XPath` 提供多种选择器，可以通过节点名称、属性、文本内容等方式精准选择元素，甚至可以选择父节点、祖先节点等。这使得在处理复杂的 HTML 结构时非常灵活。
3. **速度快**：
   - `lxml` 库（用于解析 `XPath`）是用 C 语言编写的，因此在解析大型文档时，速度非常快。相比 `BeautifulSoup`，`XPath` 通常能更快地找到指定元素。
4. **适用于结构化数据**：
   - `XPath` 非常适用于 XML 格式的数据或结构化的 HTML 数据。如果文档结构规范且稳定，使用 `XPath` 是一个高效的选择。

缺点

1. **语法复杂**：
   - `XPath` 的语法较为复杂，初学者可能需要花一些时间来学习其各种表达式和函数，尤其是在需要复杂查询时。
2. **容错性差**：
   - `XPath` 需要严格遵守文档结构，对于不完整或格式不规范的 HTML 文件，它可能无法正常工作。例如，如果文档中缺少某个标签或结构被破坏，`XPath` 查询可能会失败。
3. **依赖特定库**：
   - `XPath` 通常需要与 `lxml` 或 `xml.etree.ElementTree` 等库配合使用，而这些库在某些环境下需要额外的安装和配置。

BeautifulSoup (`bs4`)

优点

1. **简单易学**：
   - `BeautifulSoup` 的 API 设计非常直观且易于理解。只需要一些基础的 HTML 知识和简单的选择器就可以快速上手，初学者可以很快掌握。
2. **容错性强**：
   - `BeautifulSoup` 能够处理不规范的 HTML，例如标签缺失、嵌套错误等问题。它可以在解析过程中自动修复 HTML，确保能够提取数据。
3. **多种选择器**：
   - 支持多种选择器方式，包括标签选择、属性选择、CSS 选择器等。还可以通过链式调用方便地找到嵌套的元素。
4. **灵活性高**：
   - 适用于各种 HTML 文档，无论是简单还是复杂的结构都可以使用 `BeautifulSoup` 进行处理。它提供了多种查找和遍历节点的方法，适应性很强。

缺点

1. **性能较慢**：
   - `BeautifulSoup` 是纯 Python 实现的，在处理大型文档时速度较慢。相比 `lxml`，其性能不如 `XPath` 高效，尤其是在需要进行大量查询操作的情况下。
2. **处理复杂查询较麻烦**：
   - 虽然 `BeautifulSoup` 提供了很多查找和选择方法，但是在处理非常复杂的查询时，代码会变得冗长且难以维护。例如，在需要通过多重条件筛选元素时，`XPath` 通常更为简洁。
3. **功能相对有限**：
   - `BeautifulSoup` 提供的功能虽然多样，但没有 `XPath` 那样强大的表达式功能。例如，无法轻松地选择父节点、祖先节点等，且对于多层级条件选择不如 `XPath` 灵活。

何时使用 XPath 和 BeautifulSoup

- **使用 XPath 的场景**：
  - 需要高效地处理大型文档。
  - 数据结构相对固定且规范，能够通过复杂的查询表达式定位数据。
  - 需要对元素进行复杂的筛选、过滤和多级嵌套查询。
- **使用 BeautifulSoup 的场景**：
  - HTML 文档格式不规范（如一些不完整的网页）。
  - 初学者或希望快速上手进行简单数据提取的场景。
  - 需要高容错性，并且对性能要求不高。

总结

- 如果你希望高效、精确地处理结构化的 HTML/XML，并且对性能有较高的要求，可以使用 `XPath`。
- 如果你处理的是格式不规范的 HTML，或者希望更简便地编写代码，`BeautifulSoup` 是更合适的选择。